# Arthena Data Science Interview Task:

# John Elton

###  In the code below I attempt to address the problem of predicting the 'hammer price' of lots. The general approach is to train a regression model for predicting this price given the features in the data sets provided. The questions for parts 1 and 2 are answered below and are followed by the code used and various outputs produced (along with additional comments about the code along the way). The final questions are answered at the end.

# Part 1:

### Did you perform any data cleaning, filtering or transformations to improve the model fit?

### Why did you choose this model? 

### What loss function did you use? 

### Why did you pick this loss function? 

### Describe how your model is validated. What error metrics did you evaluate your model against? 

### How does the time-series nature of your data affect how you trained and tested your model?


# Part 2:

### a. Briefly discuss your approach to this problem. Pick a few of the lots that you generated `hammer_price` predictions for and do this. Do any of the distributions follow a familiar form? How does the variance differ between plots?

###  b. Compare your `hammer_price` distributions to the global distribution of all `hammer_price`s. You may consider normalizing them by dividing by one of the estimates. How do they diverge from the global distribution? How confident are you in your results?


# Additional Questions

### 1.)  Which features are most important for your model? Are there any features that surprised you? Given more data, describe other features you feel would be useful in improving the accuracy of your model.

From the


### 2.)  Suppose we're tasked with evaluating the price of a work where we're given an incomplete set of features. Say, we're given the artist name, type of work, and size, but we know nothing else. How would you go about estimating the `hammer_price` at an auction tomorrow?



### 3.)  Assume we care much more about not overpredicting `hammer_price` than we do about underpredicting the price. Describe how you would go about changing your solution in terms of the model, objective function, etc.


### 4.)  Given more time, but no new features, how much do you think you could improve the accuracy of your models by? Why?


### 5.)  Was this fun? Which sections / questions were the most difficult and which were the easiest?

